# Phase 3: Supervised Solution（监督学习深度优化）

---

## 📋 本阶段目标

在Phase 2基线模型的基础上，通过**高级特征工程、精细预处理、超参数调优**来显著提升模型性能。

### 核心任务

1. ✅ **回顾Phase 2基线性能**
2. ✅ **高级特征工程**：交互特征、多项式特征、特征选择
3. ✅ **高级数据预处理**：KNN填充、迭代填充、高级编码
4. ✅ **超参数调优**：网格搜索/随机搜索
5. ✅ **模型诊断**：学习曲线、特征重要性、错误分析
6. ✅ **模型集成**（可选）：Voting、Stacking
7. ✅ **性能对比**：Phase 2 vs Phase 3

### 方法论

- **数据驱动**：基于Phase 1的数据诊断结果决定优化策略
- **迭代优化**：逐步尝试不同的优化方法，观察性能变化
- **避免过拟合**：使用验证集监控，防止优化过度
- **可解释性**：关注特征重要性，理解模型决策

---

## 预期时间

⏱️ **30-90分钟**（取决于数据集大小和调优深度）

---

## 1. 环境准备

In [ ]:
# 导入必要的库
import sys
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 导入本项目的模块
from src import data_preprocessing as dp
from src import supervised_pipeline as sp
from src import model_evaluation as me
from src import feature_engineering as fe
from src import visualization as viz

# 设置
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 随机种子
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ 环境准备完成")
print(f"当前工作目录: {Path.cwd()}")

---

## 2. 数据加载与Phase 2回顾

### 2.1 加载数据

In [ ]:
# 加载原始数据（与Phase 2相同）
df = sns.load_dataset('titanic')
target_col = 'survived'

print(f"数据形状: {df.shape}")
print(f"目标变量: {target_col}")

# 记录原始数据信息用于后续对比
original_shape = df.shape
df.head()

### 2.2 Phase 2 基线回顾

我们先用Phase 2的简单预处理方法建立基线，作为对比基准

In [ ]:
# Phase 2 简单预处理（用于对比）
print("="*60)
print("Phase 2 基线预处理（快速方法）")
print("="*60)

X_baseline, y_baseline = dp.quick_preprocess(
    df.copy(), 
    target_col=target_col,
    drop_missing_threshold=0.5,
    handle_categorical=True,
    scale_features=True
)

print(f"\nPhase 2 基线特征数: {X_baseline.shape[1]}")

In [ ]:
# 划分数据集（保持一致性）
X_train_base, X_val_base, X_test_base, y_train_base, y_val_base, y_test_base = dp.split_data(
    X_baseline, y_baseline,
    train_size=0.7,
    val_size=0.15,
    test_size=0.15,
    random_state=RANDOM_STATE,
    stratify=True
)

print(f"训练集: {X_train_base.shape}")
print(f"验证集: {X_val_base.shape}")
print(f"测试集: {X_test_base.shape}")

In [ ]:
# Phase 2 基线模型性能（快速训练一个模型作为参考）
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

baseline_model = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
baseline_model.fit(X_train_base, y_train_base)

y_val_pred_base = baseline_model.predict(X_val_base)
baseline_val_acc = accuracy_score(y_val_base, y_val_pred_base)
baseline_val_f1 = f1_score(y_val_base, y_val_pred_base, average='binary')

print("\n" + "="*60)
print("Phase 2 基线性能（RandomForest）")
print("="*60)
print(f"验证集 Accuracy: {baseline_val_acc:.4f}")
print(f"验证集 F1-Score: {baseline_val_f1:.4f}")
print("="*60)

# 保存基线性能用于后续对比
baseline_metrics = {
    'accuracy': baseline_val_acc,
    'f1': baseline_val_f1,
    'n_features': X_baseline.shape[1]
}

---

## 3. Phase 3 高级预处理

### 3.1 高级缺失值处理

Phase 2使用简单填充（中位数/众数），Phase 3使用更智能的方法

In [ ]:
# 重新加载数据，进行高级预处理
df_advanced = df.copy()

# 先分离目标变量
y = df_advanced[target_col]
X = df_advanced.drop(columns=[target_col])

print("="*60)
print("Phase 3 高级预处理")
print("="*60)

# 删除明显无用的列
drop_cols = []
for col in X.columns:
    # 删除唯一值过多的列（ID列）
    if X[col].nunique() / len(X) > 0.95:
        drop_cols.append(col)
        
if drop_cols:
    print(f"\n删除ID列: {drop_cols}")
    X = X.drop(columns=drop_cols)

# 删除缺失率过高的列（>50%）
missing_rate = X.isnull().sum() / len(X)
high_missing_cols = missing_rate[missing_rate > 0.5].index.tolist()

if high_missing_cols:
    print(f"删除高缺失率列: {high_missing_cols}")
    X = X.drop(columns=high_missing_cols)

print(f"\n预处理后形状: {X.shape}")

In [ ]:
# 分离数值型和类别型特征
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"数值型特征: {len(numeric_cols)} 个")
print(f"  {numeric_cols}")
print(f"\n类别型特征: {len(categorical_cols)} 个")
print(f"  {categorical_cols}")

In [ ]:
# 高级缺失值填充：使用KNN填充数值特征
print("\n" + "="*60)
print("使用KNN填充数值特征的缺失值")
print("="*60)

# 检查数值特征的缺失情况
numeric_missing = X[numeric_cols].isnull().sum()
numeric_missing_cols = numeric_missing[numeric_missing > 0].index.tolist()

if numeric_missing_cols:
    print(f"有缺失值的数值特征: {numeric_missing_cols}")
    X = dp.knn_impute(X, columns=numeric_missing_cols, n_neighbors=5)
    print("✅ KNN填充完成")
else:
    print("✓ 数值特征无缺失值")

In [ ]:
# 类别特征的缺失值填充（使用众数）
print("\n" + "="*60)
print("填充类别特征的缺失值")
print("="*60)

for col in categorical_cols:
    if X[col].isnull().sum() > 0:
        mode_value = X[col].mode()[0] if len(X[col].mode()) > 0 else 'Unknown'
        X[col] = X[col].fillna(mode_value)
        print(f"  {col}: 填充为 '{mode_value}'")

print("\n✅ 缺失值处理完成")
print(f"剩余缺失值: {X.isnull().sum().sum()}")

### 3.2 高级编码

对于高基数类别特征，使用更智能的编码方法

In [ ]:
# 检查类别特征的基数
print("类别特征基数分析:")
for col in categorical_cols:
    n_unique = X[col].nunique()
    print(f"  {col}: {n_unique} 个唯一值")

# 分类处理
low_cardinality_cols = []   # 类别数 <= 10: One-Hot编码
high_cardinality_cols = []  # 类别数 > 10: 频率编码

for col in categorical_cols:
    if X[col].nunique() <= 10:
        low_cardinality_cols.append(col)
    else:
        high_cardinality_cols.append(col)

print(f"\n低基数特征（One-Hot）: {low_cardinality_cols}")
print(f"高基数特征（频率编码）: {high_cardinality_cols}")

In [ ]:
# One-Hot编码低基数特征
if low_cardinality_cols:
    print("\n" + "="*60)
    print("One-Hot编码低基数类别特征")
    print("="*60)
    
    X = dp.onehot_encode(X, columns=low_cardinality_cols, drop_first=True)
    print("✅ One-Hot编码完成")

# 频率编码高基数特征
if high_cardinality_cols:
    print("\n" + "="*60)
    print("频率编码高基数类别特征")
    print("="*60)
    
    X = dp.frequency_encoding(X, columns=high_cardinality_cols, normalize=True)
    print("✅ 频率编码完成")

print(f"\n编码后特征数: {X.shape[1]}")

### 3.3 特征缩放

In [ ]:
# 标准化数值特征
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()

if numeric_features:
    print("\n" + "="*60)
    print("标准化数值特征")
    print("="*60)
    
    X = dp.standardize_features(X, columns=numeric_features)
    print(f"✅ 标准化 {len(numeric_features)} 个数值特征")

---

## 4. 高级特征工程

### 4.1 创建交互特征

对于重要的数值特征，创建交互项来捕捉非线性关系

In [ ]:
# 选择一些重要的数值特征进行交互
# 这里手动选择，实际应用中可以基于Phase 1的相关性分析
important_numeric_cols = [col for col in numeric_features if not col.startswith(('PCA', 'row_'))]
important_numeric_cols = important_numeric_cols[:3]  # 限制为前3个，避免特征爆炸

if len(important_numeric_cols) >= 2:
    print("="*60)
    print(f"创建交互特征: {important_numeric_cols}")
    print("="*60)
    
    X_interaction = fe.create_interaction_features(
        X, 
        columns=important_numeric_cols,
        operations=['*', '+']  # 乘法和加法交互
    )
    
    # 只保留新增的交互特征
    new_cols = [col for col in X_interaction.columns if col not in X.columns]
    X = pd.concat([X, X_interaction[new_cols]], axis=1)
    
    print(f"\n✅ 新增 {len(new_cols)} 个交互特征")
    print(f"当前特征数: {X.shape[1]}")
else:
    print("数值特征不足，跳过交互特征创建")

### 4.2 多项式特征（可选）

对于非线性关系明显的特征，创建多项式特征

In [ ]:
# 注意：多项式特征会快速增加特征数量，这里仅作演示
# 实际使用时需要谨慎选择特征和多项式次数

# poly_cols = important_numeric_cols[:2]  # 只选2个特征

# if len(poly_cols) >= 1:
#     print("="*60)
#     print(f"创建多项式特征 (degree=2): {poly_cols}")
#     print("="*60)
#     
#     X = fe.create_polynomial_features(
#         X,
#         columns=poly_cols,
#         degree=2,
#         interaction_only=False
#     )
#     
#     print(f"\n✅ 多项式特征创建完成")
#     print(f"当前特征数: {X.shape[1]}")

print("⏭️  跳过多项式特征（避免特征爆炸）")

### 4.3 特征选择

移除冗余和低信息量的特征

In [ ]:
# 方差过滤（移除低方差特征）
print("="*60)
print("特征选择：方差过滤")
print("="*60)

X = fe.select_by_variance(X, threshold=0.01)
print(f"方差过滤后特征数: {X.shape[1]}")

In [ ]:
# 相关性过滤（移除高度相关的特征）
print("\n" + "="*60)
print("特征选择：相关性过滤")
print("="*60)

X = fe.select_by_correlation(X, threshold=0.90)
print(f"相关性过滤后特征数: {X.shape[1]}")

In [ ]:
# 最终检查
print("\n" + "="*60)
print("Phase 3 特征工程总结")
print("="*60)
print(f"原始特征数: {original_shape[1] - 1}")
print(f"Phase 2 特征数: {baseline_metrics['n_features']}")
print(f"Phase 3 特征数: {X.shape[1]}")
print(f"\n最终数据形状: {X.shape}")
print(f"缺失值总数: {X.isnull().sum().sum()}")

---

## 5. 数据集划分

使用与Phase 2相同的划分方式以保证可比性

In [ ]:
# 划分数据集
X_train, X_val, X_test, y_train, y_val, y_test = dp.split_data(
    X, y,
    train_size=0.7,
    val_size=0.15,
    test_size=0.15,
    random_state=RANDOM_STATE,
    stratify=True
)

print("数据集划分:")
print(f"  训练集: {X_train.shape}")
print(f"  验证集: {X_val.shape}")
print(f"  测试集: {X_test.shape}")

---

## 6. 模型训练与对比

### 6.1 使用优化后的特征训练基础模型

In [ ]:
# 确定问题类型
problem_type = 'classification' if y.nunique() <= 20 else 'regression'
print(f"问题类型: {problem_type}")

# 创建Pipeline并训练
pipeline_improved = sp.SupervisedPipeline(
    problem_type=problem_type,
    random_state=RANDOM_STATE
)

# 训练多个模型
pipeline_improved.fit(X_train, y_train, cv=5)

In [ ]:
# 模型对比
comparison_df = pipeline_improved.get_model_comparison()

print("\n" + "="*60)
print("Phase 3 模型性能对比")
print("="*60)
print(comparison_df.to_string(index=False))

In [ ]:
# 验证集评估
y_val_pred = pipeline_improved.predict(X_val)

if problem_type == 'classification':
    y_val_proba = pipeline_improved.predict_proba(X_val)
    
    val_metrics_improved = me.evaluate_classification(
        y_val,
        y_val_pred,
        y_val_proba[:, 1] if y_val_proba.shape[1] == 2 else None,
        verbose=True
    )
else:
    val_metrics_improved = me.evaluate_regression(
        y_val,
        y_val_pred,
        verbose=True
    )

### 6.2 Phase 2 vs Phase 3 性能对比

In [ ]:
# 性能对比
print("\n" + "="*60)
print("Phase 2 vs Phase 3 性能对比")
print("="*60)

comparison_data = pd.DataFrame({
    'Phase': ['Phase 2 (基线)', 'Phase 3 (优化)'],
    'Accuracy': [
        baseline_metrics['accuracy'],
        val_metrics_improved['accuracy']
    ],
    'F1-Score': [
        baseline_metrics['f1'],
        val_metrics_improved['f1']
    ],
    'N_Features': [
        baseline_metrics['n_features'],
        X.shape[1]
    ]
})

print(comparison_data.to_string(index=False))

# 计算提升
acc_improvement = (val_metrics_improved['accuracy'] - baseline_metrics['accuracy']) / baseline_metrics['accuracy'] * 100
f1_improvement = (val_metrics_improved['f1'] - baseline_metrics['f1']) / baseline_metrics['f1'] * 100

print(f"\n📈 性能提升:")
print(f"  Accuracy: {acc_improvement:+.2f}%")
print(f"  F1-Score: {f1_improvement:+.2f}%")

In [ ]:
# 可视化性能对比
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy对比
axes[0].bar(['Phase 2', 'Phase 3'], 
           [baseline_metrics['accuracy'], val_metrics_improved['accuracy']],
           color=['skyblue', 'orange'],
           edgecolor='navy',
           alpha=0.7)
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Accuracy 对比')
axes[0].set_ylim([0, 1])
axes[0].grid(axis='y', alpha=0.3)

# 添加数值标签
for i, v in enumerate([baseline_metrics['accuracy'], val_metrics_improved['accuracy']]):
    axes[0].text(i, v + 0.02, f'{v:.4f}', ha='center', fontweight='bold')

# F1-Score对比
axes[1].bar(['Phase 2', 'Phase 3'],
           [baseline_metrics['f1'], val_metrics_improved['f1']],
           color=['skyblue', 'orange'],
           edgecolor='navy',
           alpha=0.7)
axes[1].set_ylabel('F1-Score')
axes[1].set_title('F1-Score 对比')
axes[1].set_ylim([0, 1])
axes[1].grid(axis='y', alpha=0.3)

for i, v in enumerate([baseline_metrics['f1'], val_metrics_improved['f1']]):
    axes[1].text(i, v + 0.02, f'{v:.4f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

---

## 7. 超参数调优

对最佳模型进行超参数优化

In [ ]:
# 超参数网格（根据最佳模型选择）
best_model_name = pipeline_improved.best_model_name

print("="*60)
print(f"对 {best_model_name} 进行超参数调优")
print("="*60)

# 定义参数网格
param_grids = {
    'RandomForest': {
        'classifier__n_estimators': [100, 200],
        'classifier__max_depth': [10, 15, 20],
        'classifier__min_samples_split': [2, 5]
    },
    'XGBoost': {
        'classifier__n_estimators': [100, 200],
        'classifier__max_depth': [3, 5, 7],
        'classifier__learning_rate': [0.01, 0.1]
    },
    'LightGBM': {
        'classifier__n_estimators': [100, 200],
        'classifier__max_depth': [5, 7, 10],
        'classifier__learning_rate': [0.01, 0.1]
    },
    'LogisticRegression': {
        'classifier__C': [0.1, 1.0, 10.0],
        'classifier__penalty': ['l1', 'l2'],
        'classifier__solver': ['liblinear']
    }
}

# 获取对应的参数网格
param_grid = param_grids.get(best_model_name, {})

if param_grid:
    # 进行超参数调优（使用随机搜索，更快）
    tuning_results = pipeline_improved.tune_hyperparameters(
        X_train, y_train,
        param_grid=param_grid,
        cv=3,
        n_iter=10  # 随机搜索10次
    )
    
    print("\n✅ 超参数调优完成")
else:
    print(f"⚠️  {best_model_name} 没有预定义的参数网格")

In [ ]:
# 使用调优后的模型重新评估
y_val_pred_tuned = pipeline_improved.predict(X_val)

if problem_type == 'classification':
    y_val_proba_tuned = pipeline_improved.predict_proba(X_val)
    
    print("\n" + "="*60)
    print("调优后验证集性能")
    print("="*60)
    
    val_metrics_tuned = me.evaluate_classification(
        y_val,
        y_val_pred_tuned,
        y_val_proba_tuned[:, 1] if y_val_proba_tuned.shape[1] == 2 else None,
        verbose=True
    )

---

## 8. 模型诊断

### 8.1 学习曲线分析

In [ ]:
# 绘制学习曲线
print("="*60)
print("学习曲线分析（诊断过拟合/欠拟合）")
print("="*60)

me.plot_learning_curve(
    pipeline_improved.best_model,
    X_train, y_train,
    cv=3,
    scoring='accuracy'
)

### 8.2 特征重要性分析

In [ ]:
# 提取模型的特征重要性
# 注意：需要从Pipeline中提取实际的模型
try:
    # 获取Pipeline中的最后一步（模型）
    actual_model = pipeline_improved.best_model.named_steps['classifier']
    
    if hasattr(actual_model, 'feature_importances_'):
        print("="*60)
        print("特征重要性分析")
        print("="*60)
        
        # 获取转换后的特征名称
        # 由于Pipeline中包含预处理，特征名称可能已改变
        # 这里简化处理，使用原始特征名称
        feature_names = X_train.columns.tolist()
        
        me.plot_feature_importance(
            actual_model,
            feature_names,
            top_n=20
        )
    else:
        print(f"⚠️  {best_model_name} 不支持feature_importances_")
except Exception as e:
    print(f"⚠️  无法提取特征重要性: {e}")

### 8.3 混淆矩阵和ROC曲线

In [ ]:
if problem_type == 'classification':
    # 混淆矩阵
    me.plot_confusion_matrix(y_val, y_val_pred_tuned)
    plt.show()
    
    # ROC曲线（二分类）
    if y.nunique() == 2:
        me.plot_roc_curve(y_val, y_val_proba_tuned[:, 1])
        plt.show()

---

## 9. 测试集最终评估

In [ ]:
# 在测试集上评估最终模型
y_test_pred = pipeline_improved.predict(X_test)

if problem_type == 'classification':
    y_test_proba = pipeline_improved.predict_proba(X_test)
    
    print("="*60)
    print(f"测试集最终评估 - {pipeline_improved.best_model_name} (调优后)")
    print("="*60)
    
    test_metrics = me.evaluate_classification(
        y_test,
        y_test_pred,
        y_test_proba[:, 1] if y_test_proba.shape[1] == 2 else None,
        verbose=True
    )
    
    # 混淆矩阵
    me.plot_confusion_matrix(y_test, y_test_pred)
    plt.show()
else:
    print("="*60)
    print(f"测试集最终评估 - {pipeline_improved.best_model_name} (调优后)")
    print("="*60)
    
    test_metrics = me.evaluate_regression(
        y_test,
        y_test_pred,
        verbose=True
    )
    
    me.plot_regression_results(y_test, y_test_pred)
    plt.show()

---

## 10. Phase 3 总结

### 10.1 完整性能对比

In [ ]:
# 汇总所有阶段的性能
print("\n" + "="*60)
print("Phase 2 → Phase 3 完整对比")
print("="*60)

final_comparison = pd.DataFrame({
    '阶段': ['Phase 2 基线', 'Phase 3 优化', 'Phase 3 调优'],
    'Accuracy': [
        baseline_metrics['accuracy'],
        val_metrics_improved['accuracy'],
        val_metrics_tuned['accuracy']
    ],
    'F1-Score': [
        baseline_metrics['f1'],
        val_metrics_improved['f1'],
        val_metrics_tuned['f1']
    ],
    '特征数': [
        baseline_metrics['n_features'],
        X.shape[1],
        X.shape[1]
    ]
})

print(final_comparison.to_string(index=False))

# 总提升
total_acc_improvement = (val_metrics_tuned['accuracy'] - baseline_metrics['accuracy']) / baseline_metrics['accuracy'] * 100
total_f1_improvement = (val_metrics_tuned['f1'] - baseline_metrics['f1']) / baseline_metrics['f1'] * 100

print(f"\n🎯 Phase 2 → Phase 3 总提升:")
print(f"  Accuracy: {total_acc_improvement:+.2f}%")
print(f"  F1-Score: {total_f1_improvement:+.2f}%")

### 10.2 优化方法总结

In [ ]:
print("\n" + "="*60)
print("Phase 3 采用的优化方法")
print("="*60)

optimizations = [
    "1. 高级缺失值处理",
    "   ✅ KNN填充数值特征",
    "   ✅ 智能填充类别特征",
    "",
    "2. 高级编码策略",
    "   ✅ 低基数特征: One-Hot编码",
    "   ✅ 高基数特征: 频率编码",
    "",
    "3. 特征工程",
    f"   ✅ 交互特征: 新增{len([c for c in X.columns if 'times' in c or 'plus' in c])}个",
    "   ✅ 方差过滤: 移除低信息量特征",
    "   ✅ 相关性过滤: 移除冗余特征",
    "",
    "4. 模型优化",
    f"   ✅ 最佳模型: {pipeline_improved.best_model_name}",
    "   ✅ 超参数调优: 随机搜索",
    "",
    "5. 模型诊断",
    "   ✅ 学习曲线分析",
    "   ✅ 特征重要性分析",
    "   ✅ 混淆矩阵与ROC曲线"
]

for item in optimizations:
    print(item)

### 10.3 Phase 3 检查清单

In [ ]:
print("\n" + "="*60)
print("✅ Phase 3 完成检查清单")
print("="*60)

checklist = [
    "✅ Phase 2基线性能已建立",
    "✅ 高级缺失值处理完成（KNN填充）",
    "✅ 高级编码完成（频率编码）",
    "✅ 特征工程完成（交互、过滤）",
    "✅ 特征选择完成（方差、相关性）",
    "✅ 模型训练和对比完成",
    "✅ 超参数调优完成",
    "✅ 学习曲线分析完成",
    "✅ 特征重要性分析完成",
    "✅ 测试集最终评估完成",
    "✅ Phase 2 vs Phase 3性能对比完成"
]

for item in checklist:
    print(f"  {item}")

print("\n" + "="*60)
print("🎉 Phase 3: Supervised Solution 完成！")
print("="*60)

# 根据性能提升给出建议
if total_acc_improvement > 5:
    print("\n✅ 性能显著提升！优化策略有效。")
    print("   下一步: 前往 Phase 4 探索无监督学习方法")
elif total_acc_improvement > 0:
    print("\n✓  性能有所提升，但还有优化空间。")
    print("   建议: 尝试更多特征工程或模型集成方法")
else:
    print("\n⚠️  性能未提升，可能原因：")
    print("   - 基线已经很好，优化空间有限")
    print("   - 特征工程不适合当前数据")
    print("   - 需要更深入的数据分析")

---

## 11. 保存模型和结果（可选）

In [ ]:
# 保存优化后的模型
# import joblib
# model_path = Path('models/phase3_optimized_model.pkl')
# model_path.parent.mkdir(parents=True, exist_ok=True)
# joblib.dump(pipeline_improved.best_model, model_path)
# print(f"✅ 模型已保存到: {model_path}")

# 保存优化后的特征
# features_path = Path('data/processed/phase3_features.csv')
# features_path.parent.mkdir(parents=True, exist_ok=True)
# X.to_csv(features_path, index=False)
# print(f"✅ 特征已保存到: {features_path}")

# 保存性能对比结果
# results_path = Path('results/phase3_performance_comparison.csv')
# results_path.parent.mkdir(parents=True, exist_ok=True)
# final_comparison.to_csv(results_path, index=False)
# print(f"✅ 结果已保存到: {results_path}")